# Spatial join - EUBUCCO and OECD data

In [1]:
import psycopg2
from sshtunnel import SSHTunnelForwarder
import geopandas as gpd
import pandas as pd
from contextlib import contextmanager
import paramiko
import io
import os
from os import listdir
import numpy as np


## Import OECD FUA boundaries

In [2]:
path_folder = 'C:/Users/charl/OneDrive/Bureau/EUBUCCO/'
path_FUA_OECD = path_folder + "Data/FUA_OECD/"
oecd_countries = listdir(path_FUA_OECD)
FUA_OECD = gpd.read_file(path_FUA_OECD + "AUT/AUT_core_commuting.shp")
FUA_OECD = FUA_OECD.to_crs('epsg:3035')
for country in oecd_countries[1:24]:
    df2 = gpd.read_file(path_FUA_OECD + country + "/" +
                        country + "_core_commuting.shp")
    df2 = df2.to_crs('epsg:3035')
    FUA_OECD = gpd.GeoDataFrame(pd.concat([FUA_OECD, df2], ignore_index=True))

## Connect to server

In [6]:
with open('<ssh-key>', 'r') as f:
    key = f.read()
    pkey = paramiko.RSAKey.from_private_key(io.StringIO(key))


In [7]:
@contextmanager
def db_tunnel():
    with SSHTunnelForwarder(('194.163.151.34', 22), ssh_username='<username>', remote_bind_address=('172.30.0.4', 5432), ssh_pkey=pkey) as server:

        server.start()

        params = {
            'database': 'eubucco',
            'user': '<user>',
            'password': '<password>',
            'host': 'localhost',
            'port': server.local_bind_port,
            'connect_timeout': 10,
        }

        conn = psycopg2.connect(**params)
        conn.set_session(readonly=True)

        yield conn


## Retrieve data - spatial join

In [8]:
with db_tunnel() as conn:
    for i in range(647,648):
        print(FUA_OECD.iloc[i].fuaname)
        boundary = FUA_OECD.iloc[i].geometry
        #boundary = 'POLYGON ((3762754.4419298171997070 2890991.8044788637198508, 3764095.9635624778456986 2888187.3515811231918633, 3761960.7702519670128822 2888983.4717874415218830, 3761945.1074829874560237 2889266.3062656684778631, 3761744.4274468249641359 2890595.5182541240938008, 3762754.4419298171997070 2890991.8044788637198508))'
        id_code = FUA_OECD.iloc[i].fuacode
        sql = f"""
            select bu.id, bu.id_source, bc.name as country, br.name as region, b.name as city, bu.height, bu.age, bu.type, bu.type_source, bu.geometry
                from data_building as bu
                    RIGHT OUTER JOIN data_country bc on bu.country_id = bc.id
                    RIGHT OUTER JOIN data_region br on br.id = bu.region_id
                    RIGHT OUTER JOIN data_city b on b.id = bu.city_id
                WHERE ST_Within( bu.geometry, 'SRID=3035;{boundary}');
        """

        result = gpd.GeoDataFrame.from_postgis(
            sql=sql,
            con=conn,
            geom_col='geometry',
            index_col='id',
        )

        #results.append(result)
        result.to_file(path_folder + "Data/EUBUCCO_by_OECD_FUA_v2/" + id_code + '.shp')
        #result
    #data = pd.concat(results)


Uppsala


In [9]:
print(result)

                                  id_source country   region        city  \
id                                                                         
v0.1-SWE.16.1_1-1479   sweden-latest1508502  Sweden  Uppsala  Älvkarleby   
v0.1-SWE.16.6_1-833    sweden-latest1508307  Sweden  Uppsala       Tierp   
v0.1-SWE.16.6_1-834    sweden-latest1508308  Sweden  Uppsala       Tierp   
v0.1-SWE.16.6_1-835    sweden-latest1508309  Sweden  Uppsala       Tierp   
v0.1-SWE.16.6_1-836    sweden-latest1508310  Sweden  Uppsala       Tierp   
...                                     ...     ...      ...         ...   
v0.1-SWE.16.7_1-22913   sweden-latest616248  Sweden  Uppsala     Uppsala   
v0.1-SWE.16.7_1-22914   sweden-latest616249  Sweden  Uppsala     Uppsala   
v0.1-SWE.16.7_1-23512   sweden-latest617216  Sweden  Uppsala     Uppsala   
v0.1-SWE.16.7_1-23513   sweden-latest617217  Sweden  Uppsala     Uppsala   
v0.1-SWE.16.7_1-23929   sweden-latest629488  Sweden  Uppsala     Uppsala   

           